# 画像による物体検出

Matplotlib と Pillow のインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q matplotlib Pillow

In [ ]:
%run setup_demo_env.py

In [ ]:
import asyncio
import io

import grpc
import kachaka_api
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
from PIL import Image

client = kachaka_api.aio.KachakaApiClient(kachaka_api_server)

LABEL_NAME = ["?", "person", "shelf", "charger", "door"]
LABEL_COLOR = ["pink", "green", "blue", "cyan", "red"]

stream_i = client.front_camera_ros_compressed_image.stream()
stream_d = client.object_detection.stream()

while True:
    resp_i, resp_d = await asyncio.gather(anext(stream_i), anext(stream_d))

    clear_output(wait=True)
    fig, ax = plt.subplots(1)

    img = Image.open(io.BytesIO(resp_i.data))
    ax.imshow(img)

    objects = resp_d[1]
    for obj in objects:
        x, y, w, h = obj.roi.x_offset, obj.roi.y_offset, obj.roi.width, obj.roi.height
        rect = patches.Rectangle(
            (x, y),
            w,
            h,
            linewidth=2,
            edgecolor=LABEL_COLOR[obj.label],
            facecolor="none",
        )
        ax.add_patch(rect)
        label = f"{LABEL_NAME[obj.label]}, score={obj.score:.2f}"
        plt.text(x, y + h, label, color="white", backgroundcolor="grey")

    plt.axis("off")
    plt.show()